# Sparkify - Cassandra
ETL (Extract, Transform & Load) Pipline to extract event data from csv files and ingestion into an Apache Cassandra Database.  
This Project is part of the Udacity Data Engineering nanodegree.

## Aim
This project was created for Sparkify, a music streaming provider.
The Analytics team requires a high availability database with low latency capable of handling large amounts of data whilst providing fast reads for the following types of queries:

1. artist, song title and song length for sessionId 338 and itemInSession 4
2. artist, song title, first name and last name of user with the id 10 in session 182
3. every user first name and last name who listend to 'All Hands Against His Own'

## Solution

- Pre-processing the data (into a single file)
- Connecting to the cluster & creating the keyspace
- For every Query:
  - Design NoSQL data model & create table
  - Ingest Data
  - Verify Data & Table Design

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
from cassandra.cluster import Cluster
import re
import os
import glob
import numpy as np
import json
import csv

### Pre-processing

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/src


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
print(len(full_data_rows_list))
# list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Connecting to the cluster & creating the keyspace

#### Creating a Cluster

In [5]:
cluster = Cluster(['sparkifycassandra'])
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = 
{'class': 'SimpleStrategy', 'replication_factor': 1}"""
)

#### Set Keyspace

In [7]:
session.set_keyspace('sparkify')

### Query 1

>Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Select statement:

```SQL
    SELECT artist_name, title, length
    FROM   session_history
    WHERE  sessionId = 338
    AND    itemInSession = 4
```

**Table Design**  
For table session_history, sessionId is used as a partition key because the queries will filter by this column and it allows to evenly spread the data among the nodes. The column itemInSession will be used as clustering columns to help make up a unique key and allow filtering.

#### Create session_history table

In [8]:
session_history_create = """
    CREATE TABLE IF NOT EXISTS session_history (
        sessionId     INT,
        itemInSession INT,
        artist_name   TEXT,
        title         TEXT,
        length        TEXT,
    PRIMARY KEY (sessionId, itemInSession)
    )
"""
try:
    session.execute(session_history_create)
except Exception as e:
    print(e)

#### ETL Event Data

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionId, itemInSession, artist_name, title, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[5]))

#### Verify Data & select query runs successfull

In [10]:
session_history_select = """
    SELECT artist_name, title, length
    FROM   session_history
    WHERE  sessionId = 338
    AND    itemInSession = 4
"""
df = pd.DataFrame(list(session.execute(session_history_select)))
df

,artist_name,title,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


>Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In the Session, with the SessionId 338 and the itemInSession 4, the user listened to the song 'Music Matters (Mark Knight Dub) by the artist 'Faithless'. The song is 495.3073 seconds long.

### Query 2
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Select statement:

```SQL
    SELECT artistName, title, userFirstName, userLastName
    FROM user_session_history
    WHERE userId = 10
    AND sessionId = 182
```

**Table Design**  
For the table user_session_history, userId and sessionId is used as a composite partition key because the queries will filter by userId and sessionId. The composite partition key improves overall performance as userId data will not be concentrated in a single node but will be spread among more than just one node. The additional clustering column itemInSession is used to create a unique key. 

#### Create user_session_history table

In [11]:
user_session_history_create = """
    CREATE TABLE IF NOT EXISTS user_session_history (
        userId         INT,
        sessionId      INT,
        itemInSession  INT,
        artistName     TEXT,
        title          TEXT,
        userFirstName  TEXT,
        userLastName   TEXT,
    PRIMARY KEY ( (userId, sessionId), itemInSession)
    )
"""

try:
    session.execute(user_session_history_create)
except Exception as e:
    print(e)

#### ETL Event Data

In [12]:
user_session_history_insert = """
    INSERT INTO user_session_history (sessionId, itemInSession, artistName, title, userId, userFirstName, userLastName)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(
            user_session_history_insert,
            (int(line[8]), int(line[3]), line[0], line[9], int(line[10]), line[1], line[4]))

#### Verify Data & select query runs successfull

In [13]:
user_session_history_select = """
    SELECT artistName, title, userFirstName, userLastName
    FROM user_session_history
    WHERE userId = 10
    AND sessionId = 182
"""

df = pd.DataFrame(list(session.execute(user_session_history_select)))
df.head()

,artistname,title,userfirstname,userlastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In the Session 182, the User Sylvie Cruz (UserId 10) has listened to "Keep On Keepin' On" by "Down To The Bone", "Greece 2000" by "Three Drives", "Kilometer" by "Sebastien Tellier" and "Catch You Baby (Steve Pitron & Max Sanna Radio Edit)" by "Lonnie Gordon".

### Query 3

> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Select statement
```SQL
    SELECT userFirstName, userLastName
    FROM song_user_history
    WHERE song = 'All Hands Against His Own'
```

**Table Design**  
For table song_user_history, song is used as a partition key because the queries will filter by this column. In order to create a unique key userId was used as an additional clustering column to create a unique Primary Key.

#### Create song_user_history table

In [14]:
album_user_history_create = """
    CREATE TABLE IF NOT EXISTS song_user_history (
        song           TEXT,
        userId         INT,
        userFirstName  TEXT,
        userLastName   TEXT,
    PRIMARY KEY (song, userId)
    )
"""
try:
    session.execute(album_user_history_create)
except Exception as e:
    print(e)


#### ETL Event Data

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_history (song, userId, userFirstName, userLastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Verify Data & select query runs successfull

In [16]:
album_user_history_select = """
    SELECT userFirstName, userLastName
    FROM song_user_history
    WHERE song = 'All Hands Against His Own'
"""
df = pd.DataFrame(list(session.execute(album_user_history_select)))
df

,userfirstname,userlastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Jacqueline Lynch, Sara Johnson and Tegan Levine have listened to 'All Hands Against His Own'.

## Cleanup

### Drop the tables before closing out the sessions

In [17]:
query1 = "DROP TABLE session_history"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

query2 = "DROP TABLE user_session_history"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

query3 = "DROP TABLE song_user_history"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()